### 4.2 네이버 데이터랩

영화 이름과 기간을 입력해 해당기간에 해당영화를 얼마나 검색했는지 확인

In [1]:
import os
import sys
import json
import urllib.request
from datetime import date
import pandas as pd
from dateutil.relativedelta import relativedelta
from tqdm import tqdm_notebook

In [2]:
movie_info_path='/home/ubuntu/miniii_su/movie/'

In [3]:
df1 = pd.read_csv(movie_info_path+'data/naver_movie_all.csv')
start_date = date(2017,1,1)
end_date = date(2017,12,31)
len(df1)

304

In [4]:
# m_name = df1['movieNm'][0]
# m_code = df1['movieCd'][0]
# n_code = df1['naver_movieCd'][0]
# m_code

20168323

In [5]:
# with open(movie_info_path+"/raw_data/movie_info/"+str(m_code)+".json") as data_file:    
#     data = json.load(data_file)
# openDt = data['movieInfoResult']['movieInfo']['openDt']
# openDt

'20180131'

In [6]:
# date(int(str(openDt)[0:4]),int(str(openDt)[4:6]),int(str(openDt)[6:8]))
# edt = date(int(str(openDt)[0:4]),int(str(openDt)[4:6]),int(str(openDt)[6:8])) + relativedelta(months=1)
# sdt = date(int(str(openDt)[0:4]),int(str(openDt)[4:6]),int(str(openDt)[6:8]))
# str(sdt)

'2018-01-31'

In [7]:
# type(str(n_code))

str

### 1) naver_movie_all.csv 파일을 불러와서 
### 영화별 검색량을 상대적인 값으로 추출

In [13]:
for i in tqdm_notebook(range(len(df1['naver_movieCd']))):
    m_name = df1['movieNm'][i]
    m_code = df1['movieCd'][i]
    n_code = df1['naver_movieCd'][i]
    
    with open(movie_info_path+'raw_data/movie_info/'+str(m_code)+'.json') as data_file:    
        data = json.load(data_file)
    openDt = data['movieInfoResult']['movieInfo']['openDt']
    
    idt = date(2016,1,1)
    sdt = date(int(str(openDt)[0:4]),int(str(openDt)[4:6]),int(str(openDt)[6:8]))
    edt = sdt + relativedelta(months=1)
    
    if(sdt > idt):
        client_id = "nA8Iwm4HJ6EM2W1dueU9"
        client_secret = "HaEPQCPbyQ"
        url = "https://openapi.naver.com/v1/datalab/search"

        body='{\"startDate\":\"'+str(sdt)+'\",\"endDate\":\"'+str(edt)+'\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\"영화\",\"keywords\":[\"'+m_name+'\"]}]}'

#         print(body)
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        request.add_header("Content-Type","application/json")
        response = urllib.request.urlopen(request, data=body.encode("utf-8"))
        rescode = response.getcode()
        
        if(rescode==200):
            response_body = response.read()
            d = json.loads(response_body)
            
        else:
            print("Error Code:" + rescode)
            
        d['results'][0]['data']
        df = pd.DataFrame(d['results'][0]['data'])
        df.to_csv(movie_info_path+'raw_data/datalab/'+str(n_code)[:-2]+'.csv',index=False)
        
    else:
        pass

### 2)  
movie_lis2 에 네이버 코드 넣기.. 3-6에서 씀 지금은 노노

In [16]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import sys
import os
import pandas as pd
from tqdm import tqdm_notebook
import multiprocessing

af='after'
be='before'
lis = pd.read_csv(movie_info_path+'data/naver_movie_all.csv')
j=0
movie_lis2=[]
for i in range(len(lis['naver_movieCd'])):
    if(os.path.isfile(movie_info_path+'raw_data/after/'+str(lis['naver_movieCd'][i])[:-2]+'.csv') == True):
        movie_lis2.insert(j,str(lis['naver_movieCd'][i])[:-2])
        j+=1

### 2) 

In [17]:
from tqdm import tqdm_notebook
df1 = pd.read_csv(movie_info_path+'data/naver_movie_all.csv')

for i in tqdm_notebook(range(len(df1))): #range(len(df1)):
    ncd = str(df1['naver_movieCd'][i])[:-2]
#     print(ncd)
    try:
        df2 = pd.read_csv(movie_info_path+'data/'+'datalab_'+ncd+'.csv')
        df3 = pd.read_csv(movie_info_path+'data/after/after_rating/'+ncd+'_rating.csv')
    
        for i in range(len(df3)):
            df3['0'][i] = str(df3['0'][i])[0:4]+"-"+str(df3['0'][i])[4:6]+"-"+str(df3['0'][i])[6:8]
        df3.rename(columns={"0":"period"}, inplace = True)
        df4 = pd.merge(df2,df3,how='left',on='period')
        for i in range(len(df4)):
            df4['1'][i] = df4['1'][i] * 100
            df4['2'][i] = df4['2'][i] * 10
        df4.to_csv('/home/ubuntu/movie/data/merge/'+ncd+'_merge.csv',index=False)
    except FileNotFoundError:
        pass